In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import os
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext

# SPARK CONTEXT

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages datastax:spark-cassandra-connector:2.0.6-s_2.11 --conf spark.cassandra.connection.host=172.17.0.2 pyspark-shell'

conf = SparkConf()
conf.setMaster("local[2]")
conf.setAppName("Cassandra crud test")

sc = SparkContext(conf=conf)
sql = SQLContext(sc)

# REGISTER EVENTS TABLE

events = sql.read.format("org.apache.spark.sql.cassandra").load(keyspace="sea", table="events")
events.registerTempTable("events")

# RESULTSETS
category_names = sql.sql("select category_name from events where category_name is not null group by category_name").toPandas()

app = dash.Dash()

app.layout = html.Div(children=[
    html.H1(children='Eventos de Segurança Pública'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),
    dcc.Dropdown(id='events-state-category',
                options=[{'label': i, 'value': i} for i in category_names['category_name']],
                value=category_names['category_name'][0]),
    dcc.Graph(id='events-state-graph')
])

@app.callback(
    dash.dependencies.Output('events-state-graph', 'figure'),
    [dash.dependencies.Input('events-state-category', 'value')])
def update_graph(events_state_category):
    events_state = sql.sql("select * from (select count(0) total, regexp_replace(state,'State of ','') rep_state from events where country = 'Brazil' and state is not null and category_name = '"+events_state_category+"' group by rep_state) order by total desc").toPandas()
    return {
        'data': [
            {'x': events_state['rep_state'], 'y': events_state['total'], 'type': 'bar', 'name': 'Sender'},
        ],
        'layout': {
            'title': 'Eventos por Estados'
        }
    }


if __name__ == '__main__':
    app.run_server(debug=False,port=8050, host='172.17.0.3')

In [ ]:
 print([{'label': row, 'value': row} for row in category_names])

In [ ]:
 print([{'label': i, 'value': i} for i in category_names['category_name']])